In [1]:
#PyData imports
import numpy as np

#Torch-related imports
import torch

In [3]:
#PyTorch settings
torch.manual_seed(0)
print('cuda device available?: ', torch.cuda.is_available())
active_device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if torch.cuda.is_available():
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
torch.set_printoptions(precision = 8)

cuda device available?:  False


In [8]:
#Neural SDE parameters
dt_flow = 1.0 #Increased from 0.1 to reduce memory.
t = 1000 #In hours.
n = int(t / dt_flow) + 1
t_span = np.linspace(0, t, n)
t_span_tensor = torch.reshape(torch.Tensor(t_span), [1, n, 1]).to(active_device) #T_span needs to be converted to tensor object. Additionally, facilitates conversion of I_S and I_D to tensor objects.
state_dim_SCON = 3 #Not including CO2 in STATE_DIM, because CO2 is an observation.
state_dim_SAWB = 4 #Not including CO2 in STATE_DIM, because CO2 is an observation.

#SBM temperature forcing parameters
temp_ref = 283.
temp_rise = 5. #High estimate of 5 celsius temperature rise by 2100.

In [9]:
print(type(t_span_tensor))
print(type(temp_ref))

<class 'torch.Tensor'>
<class 'float'>


In [10]:
def temp_gen(t: torch.Tensor, TEMP_REF: float, TEMP_RISE: float = 5) -> torch.Tensor:
    '''
    Temperature function to force soil biogeochemical models.
    This particular temperature function assumes soil temperatures will increase by TEMP_REF over the next 80 years.    
    Returns a tensor of one or more temperatures in K.
    '''
    temp = TEMP_REF + (TEMP_RISE * t) / (80 * 24 * 365) + 10 * torch.sin((2 * np.pi / 24) * t) + 10 * torch.sin((2 * np.pi / (24 * 365)) * t)
    return temp

In [11]:
temp_gen(t_span_tensor, temp_ref, temp_rise)

tensor([[[283.00000000],
         [285.59536743],
         [288.01434326],
         ...,
         [284.56951904],
         [282.50381470],
         [280.92001343]]])

In [19]:
def arrhenius_temp_dep(parameter, temp: float, Ea: torch.Tensor, TEMP_REF: float) -> torch.Tensor:
    '''
    Arrhenius temperature dependence function.
    Accepts input parameter as torch.Tensor or Python float type.
    Accepts Ea as torch.Tensor type only.
    0.008314 is the gas constant. Temperatures are in K.
    Returns a tensor of transformed parameter value(s).    
    '''
    decayed_parameter = parameter * torch.exp(-Ea / 0.008314 * (1 / temp - 1 / TEMP_REF))
    return decayed_parameter

In [20]:
print(arrhenius_temp_dep(torch.Tensor([40, 45]), temp_ref - 20, torch.Tensor([50, 33]), temp_ref))
print(arrhenius_temp_dep(35, temp_ref - 20, torch.Tensor([45]), temp_ref))

tensor([ 7.94748545, 15.48832893])
tensor([8.17374134])


In [22]:
def linear_temp_dep(parameter, temp: float, Q: torch.Tensor, TEMP_REF: float) -> torch.Tensor:
    '''
    For a parameter with linear temperature dependence, returns the transformed parameter value.
    Accepts input parameter as torch.Tensor or Python float type.    
    Q is the slope of the temperature dependence and is a varying parameter.
    Temperatures are in K.
    '''
    modified_parameter = parameter - Q * (temp - TEMP_REF)
    return modified_parameter

In [23]:
linear_temp_dep(33, temp_ref - 20, torch.Tensor([0.002]), temp_ref)

tensor([33.04000092])